In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision.ops import masks_to_boxes
import matplotlib.pyplot as plt

In [2]:
data_dir= r"/scratch/kj1447/gracelab/dataset"
image_dir = os.path.join(data_dir, 'image_stack')
mask_dir = os.path.join(data_dir, 'mask')
image_filenames = os.listdir(image_dir)
box_size = 64

In [3]:
new_image_dir = os.path.join(data_dir, 'image_stack_cropped')
new_mask_dir = os.path.join(data_dir, 'mask_cropped')
if not os.path.isdir(new_image_dir):
    os.mkdir(new_image_dir)
    print('image folder created')
if not os.path.isdir(new_mask_dir):
    os.mkdir(new_mask_dir)
    print('mask folder created')

In [4]:
len(image_filenames)

26522

In [5]:
result={}
for i, image_filename in enumerate(image_filenames):
    image_path = os.path.join(image_dir, image_filename)
    mask_filename = image_filename
    mask_path = os.path.join(mask_dir, mask_filename)
    image = np.load(image_path)['arr_0']
    
    if image.shape[0] in result:
        result[image.shape[0]]+=1
    else:
        result[image.shape[0]]=1
        
    mask = np.load(mask_path)['arr_0']
    mask[mask != 3] = 0
    mask[mask == 3] = 1
    
#     print("MASK SUM", mask.sum())
    image = torch.from_numpy(image) #kx13x228x228
    mask = torch.from_numpy(mask).float().unsqueeze(0) #1x228x228
    if mask.sum()>0:#substation detected
        # get bounding box coordinates for each mask
        boxes = masks_to_boxes(mask) # n x 4 
      
        max_box = 0
        #selecting just the first box
        x1 = boxes[max_box,0].item()
        y1 = boxes[max_box,1].item()
        x2 = boxes[max_box, 2].item()
        y2 = boxes[max_box, 3].item()

        centers = [int(x1 + 0.5*np.abs(x1 - x2)), int(y1 + 0.5*np.abs(y1 - y2))]
    else:
        centers = [image.shape[2]//2, image.shape[3]//2]
    offset = box_size//2

    x1_new = centers[0] - offset
    y1_new = centers[1] - offset
    x2_new = centers[0] + offset
    y2_new = centers[1] + offset

    #checks: 
    if x1_new<0: 
        x2_new += np.abs(x1_new) #add additional width 
        x1_new=0
    elif x2_new > mask.shape[2]: 
        x1_new = x1_new - np.abs(mask.shape[2] - x2_new) 
        x2_new = mask.shape[2] 

    if y1_new < 0: 
        y2_new += np.abs(y1_new)
        y1_new = 0 
    elif y2_new > mask.shape[1]:
        y1_new = y1_new - np.abs(mask.shape[1] - y2_new) 
        y2_new = mask.shape[1] 
    
    image_cropped = image[:,:, y1_new:y2_new, x1_new:x2_new]
    mask_cropped = mask[:, y1_new:y2_new, x1_new:x2_new,]
    mask_cropped = mask_cropped.squeeze(dim=0)
    np.save(os.path.join(new_image_dir,image_filename),image_cropped)
    np.save(os.path.join(new_mask_dir,mask_filename),mask_cropped)

    
    if i%1000==0:
        print(i/len(image_filenames)*100, " completed.")
print("Image Distribution: ",result)

0.0  completed.
3.770454716838851  completed.
7.540909433677702  completed.
11.311364150516551  completed.
15.081818867355404  completed.
18.852273584194254  completed.
22.622728301033103  completed.
26.39318301787196  completed.
30.163637734710807  completed.
33.93409245154966  completed.
37.70454716838851  completed.
41.47500188522736  completed.
45.245456602066206  completed.
49.01591131890506  completed.
52.78636603574392  completed.
56.55682075258276  completed.
60.327275469421615  completed.
64.09773018626046  completed.
67.86818490309932  completed.
71.63863961993816  completed.
75.40909433677702  completed.
79.17954905361586  completed.
82.95000377045471  completed.
86.72045848729357  completed.
90.49091320413241  completed.
94.26136792097127  completed.
98.03182263781012  completed.
Image Distribution:  {5: 24593, 3: 442, 4: 673, 1: 331, 2: 483}
